## Setup RPC & deployer

In [50]:
from dotenv import load_dotenv
import os
import boa
from eth_account import Account
from web3 import Web3
import logging
import subprocess
import time

load_dotenv()
chain_handle = "base-sepolia"

PRIVATE_KEY = os.environ.get("WEB3_TESTNET_PK")
RPCs = {
    "sepolia": "https://sepolia.drpc.org",
    "base-sepolia": "https://sepolia.base.org",
    "optimism-sepolia": "https://sepolia.optimism.io",
    "arbitrum-sepolia": "https://sepolia-rollup.arbitrum.io/rpc",
}

state_dict = {}
# RPC_URL = RPCs[chain_handle]

deployer = Account.from_key(PRIVATE_KEY)

for key in RPCs.keys():
    state_dict[key] = {}
    state_dict[key]['rpc'] = RPCs[key]
    boa.set_network_env(RPCs[key])
    boa.env.add_account(deployer)
    state_dict[key]['boa'] = boa.env # oops
    state_dict[key]['w3'] = Web3(Web3.HTTPProvider(RPCs[key]))
    
for key in state_dict.keys():
    with boa.swap_env(state_dict[key]['boa']):
        print(f"Deploying with {boa.env.eoa} on {key}, id {boa.env.evm.patch.chain_id}")
        print(f"Chain balance is {state_dict[key]['w3'].eth.get_balance(boa.env.eoa)/1e18 :.3f} ETH")

Deploying with 0x73241E98090042A718f7eb1AF07FAD27ff09A3F3 on sepolia, id 11155111
Chain balance is 0.980 ETH
Deploying with 0x73241E98090042A718f7eb1AF07FAD27ff09A3F3 on base-sepolia, id 84532
Chain balance is 4.993 ETH
Deploying with 0x73241E98090042A718f7eb1AF07FAD27ff09A3F3 on optimism-sepolia, id 11155420
Chain balance is 5.000 ETH
Deploying with 0x73241E98090042A718f7eb1AF07FAD27ff09A3F3 on arbitrum-sepolia, id 421614
Chain balance is 0.998 ETH


## Deploy block oracles and main view contract

In [47]:
main_chain = 'sepolia'

for key in state_dict.keys():
    with boa.swap_env(state_dict[key]['boa']):
        if key == main_chain:
            contract_deployer = boa.load_partial("../contracts/MainnetBlockView.vy")
            contract = contract_deployer()
            print(f"Block view deployed at {contract.address} on {key}") 
        else:                  
            contract_deployer = boa.load_partial("../contracts/BlockOracle.vy")
            contract = contract_deployer(1) # commit threshold
            print(f"Block oracle deployed at {contract.address} on {key}")
        state_dict[key]['oracle'] = contract


tx broadcasted: 0xf14ec70b9d998297237c50cb46172a9467914c7abd0cdfc0768b17df3c092e00
0xf14ec70b9d998297237c50cb46172a9467914c7abd0cdfc0768b17df3c092e00 mined in block 0xad37e6eb7e2bbf9a3bd3b4b3b5ab7ad2b41ec2099a0647bfc212a964f5ffa1d6!
contract deployed at 0xc74ad0D9479086c02512FF0fc405cdD267370284
Block view deployed at 0xc74ad0D9479086c02512FF0fc405cdD267370284 on sepolia
tx broadcasted: 0x5a6156e23a6a6e34d29abaaa812300bc1d0c441ec63a768eb7e99859719e7c28
0x5a6156e23a6a6e34d29abaaa812300bc1d0c441ec63a768eb7e99859719e7c28 mined in block 0x1f335ebc2b4e1b6ee03c26a00e86a0450510431082956853c8d0abe1bf2202e0!
contract deployed at 0x37E3ba4278AE8269C5e482052bB37E358Cfb1B35
Block oracle deployed at 0x37E3ba4278AE8269C5e482052bB37E358Cfb1B35 on base-sepolia
tx broadcasted: 0x81113bbc43c1c079b3cb8b470152192024e86f42b453444351529f37fca5a772
0x81113bbc43c1c079b3cb8b470152192024e86f42b453444351529f37fca5a772 mined in block 0x267456b3f840cefedd91b237b51aca9d7c34db111cba1e4c5e71e9711936d669!
contract dep

## Setup LZ

In [51]:
from LZDeployments import LZDeployments
lz = LZDeployments()
for chain in state_dict:
   metadata = lz.get_chain_metadata(chain)["metadata"]
   dvn_data = lz.get_chain_metadata(chain)
   
   # Update state dict
   state_dict[chain].update({
       'eid': metadata['eid'],
       'endpoint': metadata['endpointV2'],
       'send_lib': metadata.get('sendUln302', 'unavailable'),
       'receive_lib': metadata.get('receiveUln302', 'unavailable'),
       'read_lib': metadata.get('readLib1002', 'unavailable'),
       'dvns': dvn_data['dvns']
   })

   # Print info
   print(f"LZ details for {chain}:")
   print(f"Chain eID: {metadata['eid']}\nEndpoint address: {metadata['endpointV2']}")
   print(f"DVNs: {len(dvn_data['dvns'])}, Read DVNs: {len(dvn_data['dvns_lzread'])}")
   print(f"Send lib: {state_dict[chain]['send_lib']}\n"
         f"Receive lib: {state_dict[chain]['receive_lib']}\n"
         f"Read lib: {state_dict[chain]['read_lib']}\n---")

LZ details for sepolia:
Chain eID: 40161
Endpoint address: 0x6EDCE65403992e310A62460808c4b910D972f10f
DVNs: 10, Read DVNs: 1
Send lib: 0xcc1ae8Cf5D3904Cef3360A9532B477529b177cCE
Receive lib: 0xdAf00F5eE2158dD58E0d3857851c432E34A3A851
Read lib: 0x908E86e9cb3F16CC94AE7569Bf64Ce2CE04bbcBE
---
LZ details for base-sepolia:
Chain eID: 40245
Endpoint address: 0x6EDCE65403992e310A62460808c4b910D972f10f
DVNs: 4, Read DVNs: 1
Send lib: 0xC1868e054425D378095A003EcbA3823a5D0135C9
Receive lib: 0x12523de19dc41c91F7d2093E0CFbB76b17012C8d
Read lib: 0x29270F0CFC54432181C853Cd25E2Fb60A68E03f2
---
LZ details for optimism-sepolia:
Chain eID: 40232
Endpoint address: 0x6EDCE65403992e310A62460808c4b910D972f10f
DVNs: 4, Read DVNs: 0
Send lib: 0xB31D2cb502E25B30C651842C7C3293c51Fe6d16f
Receive lib: 0x9284fd59B95b9143AF0b9795CAC16eb3C723C9Ca
Read lib: unavailable
---
LZ details for arbitrum-sepolia:
Chain eID: 40231
Endpoint address: 0x6EDCE65403992e310A62460808c4b910D972f10f
DVNs: 7, Read DVNs: 1
Send lib: 0x4

# I. Deployment

In [52]:
# Deploy LZ messenger on each chain

for chain in state_dict:
    if chain == main_chain:
        continue
    with boa.swap_env(state_dict[chain]['boa']):
        contract_deployer = boa.load_partial("../contracts/messengers/LZMessenger.vy")
        contract = contract_deployer(
            state_dict[chain]['endpoint'],  # endpoint on the chain
            500_000,  # default gas limit
        )
        print(f"LZ Messenger deployed at {contract.address} on {chain}")
        state_dict[chain]['messenger'] = contract


tx broadcasted: 0x855febe1caf44c6edfc5c35d571fa5c7bbe9fbdbc8f1e9228661b3634a9c145a


/Users/michael/.local/share/uv/python/cpython-3.12.6-macos-aarch64-none/lib/python3.12/functools.py:993: UserWarning: debug_traceTransaction not available! titanoboa will try hard to interact with the network, but this means that titanoboa is not able to do certain safety checks at runtime. it is recommended to switch to a node or provider with debug_traceTransaction.
  val = self.func(instance)


0x855febe1caf44c6edfc5c35d571fa5c7bbe9fbdbc8f1e9228661b3634a9c145a mined in block 0x3617c97cbc76d6e8b2c8002cfeb74ffcb248a47eaac5fefba3c8faa354d6cc42!
contract deployed at 0x351c9FcBe456efBbFbD2A39448EF2aCC94D449Ab
LZ Messenger deployed at 0x351c9FcBe456efBbFbD2A39448EF2aCC94D449Ab on base-sepolia
tx broadcasted: 0x42bc310488d5f39b7c56f480ff1843b603605b384f2fa152c43c6ccddd7f6c09
0x42bc310488d5f39b7c56f480ff1843b603605b384f2fa152c43c6ccddd7f6c09 mined in block 0xa5236b945aa469664bf245626789b43c78a84a2c08f08905a574e7516e0378e3!
contract deployed at 0xc190a9403367c0D3343C44a2e8c683E5573BE4F7
LZ Messenger deployed at 0xc190a9403367c0D3343C44a2e8c683E5573BE4F7 on optimism-sepolia
tx broadcasted: 0xa9823ea4fbf955056f0d6bd4c8dfe4059c4d64fab763c7bb56c03ef7db1893d1
0xa9823ea4fbf955056f0d6bd4c8dfe4059c4d64fab763c7bb56c03ef7db1893d1 mined in block 0xc9e2b57b2eb27542e4be6028a95899a91fc74b27514b0c01dbdc398d62820a77!
contract deployed at 0x2665d04eBF02e27A0D95cf7B491BbC9E48069aed
LZ Messenger deploye

## II. Post-deployment interactions 
## (web3py to simulate real interactions)

### 0. Prepare infra

In [65]:
def get_vyper_abi():
    command = ["vyper", "../contracts/messengers/LZMessenger.vy", "-f", "abi_python"]
    try:
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        return result.stdout
    except subprocess.CalledProcessError as e:
        return f"Error: {e.stderr}"


logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")

ABI = get_vyper_abi()
for key in state_dict.keys():
    if key == main_chain:
        continue
    address = state_dict[key]['messenger'].address
    state_dict[key]['messenger_w3'] = state_dict[key]['w3'].eth.contract(address=address, abi=ABI)

account = Web3().eth.account.from_key(deployer.key)


def send_tx_single(w3, func, acc, value=0):
    tx = func.build_transaction(
        {
            "from": account.address,
            "nonce": w3.eth.get_transaction_count(account.address),
            "value": value,
        }
    )
    tx["gas"] = int(5 * w3.eth.estimate_gas(tx))
    signed_tx = w3.eth.account.sign_transaction(tx, private_key=account.key)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    return tx_hash


def send_tx(w3, func, acc, value=0):
    success = False
    while not success:
        try:
            tx_hash = send_tx_single(w3, func, acc, value)
            success = True
        except Exception as e:
            if "replacement transaction underpriced" in str(e) or "nonce too low" in str(e):
                print(str(e), "Retrying...")
                success = False
                time.sleep(0.5)
            else:
                raise e
    return tx_hash

## Prepare Libs

In [67]:
# send lib
for key in state_dict.keys():
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]['boa']):
        func = state_dict[key]['messenger_w3'].functions.set_lz_send_lib(state_dict[key]['eid'], state_dict[key]['send_lib'])
        tx_hash = send_tx(state_dict[key]['w3'], func, account)
        print(f"Added send lib: {tx_hash.hex()} on {key}")

# receive lib
for key in state_dict.keys():
    if key == main_chain:
        continue
    with boa.swap_env(state_dict[key]['boa']):
        func = state_dict[key]['messenger_w3'].functions.set_lz_receive_lib(state_dict[key]['eid'], state_dict[key]['receive_lib'])
        tx_hash = send_tx(state_dict[key]['w3'], func, account)
        print(f"Added receive lib: {tx_hash.hex()} on {key}")


ContractCustomError: ('0xd0ecb66b', '0xd0ecb66b')

In [ ]:
from lzreadabi import lzreadlib_abi
import json

for key in state_dict.keys():
    if key == main_chain or state_dict[key]['read_lib'] == 'unavailable':
        continue
    with boa.swap_env(state_dict[key]['boa']):
        lzreadlib = state_dict[key]['w3'].eth.contract(address=state_dict[key]['read_lib'], abi=lzreadlib_abi)
        print(f"Read lib type: {lzreadlib.functions.messageLibType().call()}")
        print(f"Read lib version: {lzreadlib.functions.version().call()}")

        eid_check = [4294967295, 4294967294]
        supported_eid = None
        for eid in eid_check:
            support = lzreadlib.functions.isSupportedEid(eid).call()
            print(f"Supports {eid}: {support}")
            if support and not supported_eid:
                supported_eid = eid
        ## read lib
        read_channel = state_dict[key]['messenger_w3'].functions.LZ_READ_CHANNEL().call()
        if supported_eid and read_channel != supported_eid:
            print(f"Setting read channel to {supported_eid}")
            func = state_dict[key]['messenger_w3'].functions.set_lz_read_channel(supported_eid)
            tx_hash = send_tx(state_dict[key]['w3'], func, account)
            print(f"Set read channel: {tx_hash.hex()}")

        # read direction
        read_channel = state_dict[key]['messenger_w3'].functions.LZ_READ_CHANNEL().call()
        print(f"Read Channel: {read_channel}")
        func = state_dict[key]['messenger_w3'].functions.set_lz_send_lib(read_channel, state_dict[key]['read_lib'])
        tx_hash3 = send_tx(state_dict[key]['w3'], func, account)
        print(f"Added read lib: {tx_hash3.hex()}")

        # # receive direction
        func = state_dict[key]['messenger_w3'].functions.set_lz_receive_lib(read_channel, state_dict[key]['read_lib'])
        tx_hash4 = send_tx(state_dict[key]['w3'], func, account)
        print(f"Added receive lib: {tx_hash4.hex()}")

Read lib type: 2
Read lib version: [10, 0, 2]
Supports 4294967295: True
Supports 4294967294: False
Read Channel: 4294967295


ContractCustomError: ('0xa4ff2ec3', '0xa4ff2ec3')

## Set up DVNs

In [ ]:
# first set the delegate
# func = contract_w3.functions.set_lz_delegate(account.address)
# tx_hash = send_tx(func, account)
# logging.info(f"Added self as delegate: {tx_hash.hex()}")
def checksum(address):
    return Web3.to_checksum_address(address)


CONFIG_TYPE_READ = 1
CONFIG_TYPE_ULN = 2
# list all dvns
print("Listing all dvns")
for dvn in dvns_all:
    print(dvn)

oapp = contract.address
confirmations = 1
required_dvns = [dvn["address"] for dvn in dvns_all if dvn["id"] == "layerzero-labs"]
required_dvns = required_dvns[0:1]  # of all eid dvns we pick first one with layerzero id
optional_dvns = [dvn["address"] for dvn in dvns_all if dvn["address"] not in required_dvns]
# all other dvs are optional, with a threshold of 2 (or less is there is not enough)
optional_dvn_threshold = min(2, len(optional_dvns))  # 2 or 1

# sort dvns alphabetically (important, otherwise lz will fail!)
required_dvns.sort()
optional_dvns.sort()
required_dvns = [checksum(dvn) for dvn in required_dvns]
optional_dvns = [checksum(dvn) for dvn in optional_dvns]

print("DVNs for send/receive:")
print(f"Required: {required_dvns}")
print(f"Optional: {optional_dvns}")
print(f"Optional threshold: {optional_dvn_threshold}")
for lib in [SEND_LIB, RECEIVE_LIB]:
    func = contract_w3.functions.set_lz_uln_config(
        LZ_EID,
        oapp,
        lib,
        CONFIG_TYPE_ULN,
        confirmations,
        required_dvns,
        optional_dvns,
        optional_dvn_threshold,
    )
    tx_hash = send_tx(func, account)
    print(f"Added ULN config: {tx_hash.hex()}")
    # break

required_dvns = [
    dvn["address"]
    for dvn in dvns_all
    if dvn["id"] == "layerzero-labs" and dvn.get("lzReadCompatible", False)
]
optional_dvns = [
    dvn["address"]
    for dvn in dvns_all
    if dvn["address"] not in required_dvns and dvn.get("lzReadCompatible", False)
]
optional_dvn_threshold = min(2, len(optional_dvns))  # 2 or 1 or 0
print("DVNs for read:")
print(f"Required: {required_dvns}")
print(f"Optional: {optional_dvns}")
print(f"Optional threshold: {optional_dvn_threshold}")
required_dvns.sort()
optional_dvns.sort()
required_dvns = [checksum(dvn) for dvn in required_dvns]
optional_dvns = [checksum(dvn) for dvn in optional_dvns]
# required_dvns = []
for lib in [READ_LIB]:
    func = contract_w3.functions.set_lz_uln_config(
        read_channel,
        oapp,
        lib,
        CONFIG_TYPE_READ,
        confirmations,
        required_dvns,
        optional_dvns,
        optional_dvn_threshold,
    )
    tx_hash = send_tx(func, account)
    print(f"Added ULN-Read config: {tx_hash.hex()}")

## 1. LZ Send & Receive

In [ ]:
# add self as peer

func = contract_w3.functions.set_peer(LZ_EID, CONTRACT_ADDRESS)
tx_hash = send_tx(func, account)
logging.info(f"Added self as peer tx: {tx_hash.hex()}")

In [ ]:
gas_limit = 500_000
value = int(0.001 * 10**18)
msg = "170475436437825620930817601234267694881687829390282260281137596999800372275961"

fee = contract_w3.functions.quote_message_fee(
    LZ_EID, CONTRACT_ADDRESS, msg, _gas_limit=gas_limit, _value=value
).call()
logging.info(f"Fee: {fee}")

func = contract_w3.functions.send_message(
    _dst_eid=LZ_EID, _receiver=CONTRACT_ADDRESS, _message=msg, _gas_limit=gas_limit, _value=value
)
tx_hash = send_tx(func, account, 3 * fee)
logging.info(f"Tx: {tx_hash.hex()}")
# after we sent the message we can check it on testnet.layerzeroscan.com (use address or txid)
# shortly after, it should be pinged back (we send to ourselves, and event be fired)

## 2. LzRead

In [ ]:
# set self as read peer
# LZ_READ_CHANNEL = 4294967294
LZ_READ_CHANNEL = contract_w3.functions.LZ_READ_CHANNEL().call()
func = contract_w3.functions.set_peer(LZ_READ_CHANNEL, CONTRACT_ADDRESS)
tx_hash = send_tx(func, account)
logging.info(f"Set peer tx: {tx_hash.hex()}")

In [ ]:
# prepare reading calldata
from vyper.utils import method_id

method_str = "dummy_endpoint(uint256)"
num = 34710
calldata = method_id(method_str) + boa.util.abi.abi_encode("(uint256)", (num,))

# read the contract itself
res = contract_w3.functions.dummy_endpoint(num).call()

logging.info(f"Read: {res}")
value = int(0.001 * 10**18)
# now quote lzread fee
fee = contract_w3.functions.quote_read_fee(
    LZ_EID, CONTRACT_ADDRESS, calldata, _gas_limit=gas_limit, _value=value
).call()
logging.info(f"Fee: {fee}")

# now request read
func = contract_w3.functions.request_read(
    LZ_EID, CONTRACT_ADDRESS, calldata, _gas_limit=gas_limit, _value=value
)
tx_hash = send_tx(func, account, 3 * fee)
logging.info(f"Tx: {tx_hash.hex()}")

In [ ]:
# test calldata
import requests

print(calldata.hex())

rpc_endpoint = "https://sepolia.base.org"
# perform eth_call request to contract using calldata
r = requests.post(
    rpc_endpoint,
    json={
        "jsonrpc": "2.0",
        "method": "eth_call",
        "params": [{"to": CONTRACT_ADDRESS, "data": "0x" + calldata.hex()}, "latest"],
        "id": 1,
    },
)
response = r.json()["result"]
# print(r.json()['result'])
print(response)
contract_code = """
@external
@view
def message_as_string(_message: Bytes[128]) -> String[128]:
    message: String[128] = convert(_message, String[128])
    return message
"""
with boa.swap_env(boa.Env()):
    tmp_contract = boa.loads(contract_code)

    res = tmp_contract.message_as_string(bytes.fromhex(response[2:]))
    print(res)

In [ ]:
contract_code = """
@external
@view
def foo():
    val_1: uint128 = 123
    val_2: uint128 = 0
    val1_b16: bytes16 = convert(1, bytes16)
    val2_b16: bytes16 = convert(2, bytes16)
    # val_b32: bytes32 = convert(val_uint, bytes32)
    # a16: Bytes[16] = empty(Bytes[16])
    # a32: Bytes[32] = empty(Bytes[32])
    val_B32: Bytes[32] = concat(val1_b16, empty(bytes16))
    # # a1: bytes16 = convert(1, bytes16)
    print(val_B32)
"""
with boa.swap_env(boa.Env()):
    tmp_contract = boa.loads(contract_code)

    res = tmp_contract.foo()
    # print(res)